In [9]:
import pandas as pd 
df = pd.read_csv("sla_cases.csv")
df["created_ts"] = pd.to_datetime(df["created_ts"])
df["closed_ts"] = pd.to_datetime(df["closed_ts"])

In [10]:
print(df.columns)

Index(['case_id', 'created_ts', 'closed_ts', 'priority', 'sla_hours',
       'product', 'region'],
      dtype='str')


In [11]:
df.columns = df.columns.str.strip()

In [12]:
df["created_ts"] = pd.to_datetime(df["created_ts"])
df["closed_ts"] = pd.to_datetime(df["closed_ts"])

In [15]:
df.rename(columns={"created_date": "created_ts"}, inplace=True)

In [16]:
print(df.columns)

Index(['case_id', 'created_ts', 'closed_ts', 'priority', 'sla_hours',
       'product', 'region'],
      dtype='str')


In [17]:
required_cols = {"created_ts", "closed_ts"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

In [18]:
print(df.columns)

Index(['case_id', 'created_ts', 'closed_ts', 'priority', 'sla_hours',
       'product', 'region'],
      dtype='str')


In [19]:
current_time = pd.Timestamp.now()

df["effective_closed_ts"] = df["closed_ts"].fillna(current_time)

df["resolution_hours"] = (
    df["effective_closed_ts"] - df["created_ts"]
).dt.total_seconds() / 3600

In [20]:
df["sla_breached"] = df["resolution_hours"] > df["sla_hours"]

In [21]:
df[["case_id", "priority", "resolution_hours", "sla_hours", "sla_breached"]]

,case_id,priority,resolution_hours,sla_hours,sla_breached
0,SLA1001,High,7.500000,8,False
1,SLA1002,Medium,747.750000,24,True
2,SLA1003,High,10.500000,8,True
3,SLA1004,High,679.500000,8,True
4,SLA1005,Low,6.250000,48,False
5,SLA1006,Medium,1466.500000,24,True
6,SLA1007,High,7105.221837,8,True
7,SLA1008,Medium,7103.971837,24,True
8,SLA1009,High,756.000000,8,True
9,SLA1010,Low,6.500000,48,False


In [22]:
df.groupby("priority")["sla_breached"].mean()



priority
High      0.833333
Low       0.000000
Medium    1.000000
Name: sla_breached, dtype: float64

In [23]:
df.groupby("product")["sla_breached"].mean()

product
CRM              0.714286
Service Cloud    0.800000
Name: sla_breached, dtype: float64

In [24]:
df["sla_usage_pct"] = df["resolution_hours"] / df["sla_hours"]

at_risk = df[
    (df["closed_ts"].isna()) &
    (df["sla_usage_pct"] >= 0.8)
]

at_risk[["case_id", "priority", "sla_usage_pct"]]

,case_id,priority,sla_usage_pct
6,SLA1007,High,888.152730
7,SLA1008,Medium,295.998827
10,SLA1011,High,705.183980


In [25]:
df["sla_status"] = pd.cut(
    df["sla_usage_pct"],
    bins=[0, 0.7, 0.8, 1, float("inf")],
    labels=["OK", "Warning", "At Risk", "Breached"]
)

In [26]:
print(df[["case_id", "sla_status"]])

    case_id sla_status
0   SLA1001    At Risk
1   SLA1002   Breached
2   SLA1003   Breached
3   SLA1004   Breached
4   SLA1005         OK
5   SLA1006   Breached
6   SLA1007   Breached
7   SLA1008   Breached
8   SLA1009   Breached
9   SLA1010         OK
10  SLA1011   Breached
11  SLA1012   Breached


In [27]:
print(
    df[[
        "case_id",
        "priority",
        "sla_hours",
        "resolution_hours",
        "sla_usage_pct",
        "sla_status"
    ]]
)

    case_id priority  sla_hours  resolution_hours  sla_usage_pct sla_status
0   SLA1001     High          8          7.500000       0.937500    At Risk
1   SLA1002   Medium         24        747.750000      31.156250   Breached
2   SLA1003     High          8         10.500000       1.312500   Breached
3   SLA1004     High          8        679.500000      84.937500   Breached
4   SLA1005      Low         48          6.250000       0.130208         OK
5   SLA1006   Medium         24       1466.500000      61.104167   Breached
6   SLA1007     High          8       7105.221837     888.152730   Breached
7   SLA1008   Medium         24       7103.971837     295.998827   Breached
8   SLA1009     High          8        756.000000      94.500000   Breached
9   SLA1010      Low         48          6.500000       0.135417         OK
10  SLA1011     High          8       5641.471837     705.183980   Breached
11  SLA1012   Medium         24        719.500000      29.979167   Breached


In [28]:
print(df["sla_status"].value_counts())

sla_status
Breached    9
OK          2
At Risk     1
Warning     0
Name: count, dtype: int64


In [29]:
print(df[df["sla_status"].isin(["At Risk", "Breached"])])

    case_id          created_ts           closed_ts priority  sla_hours  \
0   SLA1001 2025-01-01 09:00:00 2025-01-01 16:30:00     High          8   
1   SLA1002 2025-01-01 10:15:00 2025-02-01 14:00:00   Medium         24   
2   SLA1003 2025-02-01 09:30:00 2025-02-01 20:00:00     High          8   
3   SLA1004 2025-02-01 11:00:00 2025-03-01 18:30:00     High          8   
5   SLA1006 2025-03-01 09:30:00 2025-05-01 12:00:00   Medium         24   
6   SLA1007 2025-04-01 10:00:00                 NaT     High          8   
7   SLA1008 2025-04-01 11:15:00                 NaT   Medium         24   
8   SLA1009 2025-05-01 09:00:00 2025-06-01 21:00:00     High          8   
10  SLA1011 2025-06-01 09:45:00                 NaT     High          8   
11  SLA1012 2025-06-01 11:00:00 2025-07-01 10:30:00   Medium         24   

          product region        effective_closed_ts  resolution_hours  \
0             CRM   APAC 2025-01-01 16:30:00.000000          7.500000   
1   Service Cloud   EMEA 202